In [1]:
import torch
import gc

gc.collect()

torch.cuda.empty_cache()

C:\Users\thithilab\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pathlib import Path
import cv2
import numpy as np
from datetime import datetime
import os
import time
from ultralytics.yolo.utils.files import increment_path
import torch
import gc
import pickle
import pandas as pd
from tensorflow.keras.applications.vgg16 import VGG16
#from pathlib import Path
gc.collect()
torch.cuda.empty_cache()

from ultralytics.yolo.utils.plotting import Annotator

from ultralytics import YOLO

model = YOLO('runs/segment/train4/weights/best.pt')  # load a custom model

yolo->engine->model.py93


In [3]:
le_filename = '../SUMIYOSHI_FEATURES/GENERAL_LE_SUMIYOSHI.le'
predictor_filename ='../SUMIYOSHI_FEATURES/VGG_SVM_SUMIYOSHI_v1.pkl'

predictor = pickle.load(open(predictor_filename, 'rb'))
lable_encoder = pickle.load(open(le_filename, 'rb'))

In [4]:
SIZE=224

vgg = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in vgg.layers:
	layer.trainable = False
    
#vgg.summary()  #Trainable parameters will be 0


In [5]:
def Predict_SVM(image):  ## new with vgg
    
    input_img = np.expand_dims(image, axis=0) #Expand dims so the input is (num images, x, y, c)
    input_img_feature=vgg.predict(input_img)
    input_img_features=input_img_feature.reshape(input_img_feature.shape[0], -1)
    predicted_result = predictor.predict(input_img_features)[0] 
    predicted_result = lable_encoder.inverse_transform([predicted_result])  #Reverse the label encoder to original name
    
    return predicted_result
print("defined predictor")

defined predictor


In [6]:


def check_withinROI_NEW(x1,y1,x2,y2,h,w):
    #print(x1, '  ',y1, '  ',x2, '  ',y2, '  ',h, '  ',w)
    if(x1<(X1) or x2>X2 or y1<Y1_NEW or
       y2>Y2_NEW or x1>=X2):
        return False
    if(y2 - y1>2800 or y2-y1<1200): #3000 to 800 Before
        return False
    #print('IN')
    return True  

def check_withinROI_Resize(x1,y1,x2,y2,h,w):
    #print(x1, '  ',y1, '  ',x2, '  ',y2, '  ',h, '  ',w)
    resize_x1=850#*(w/2992)
    resize_x2=1050#*(w/2992)
    resize_y1=Y1_NEW#Y1_NEW*(w/2992)
    resize_y2=Y1_NEW#Y2_NEW*(w/2992)
    #print(resize_x1, '  ',resize_y1, '  ',resize_x2, '  ',resize_y2)
    if(x1<int(resize_x1) or x2>int(resize_x2) or x1>=int(resize_x2)):
        return False
    if(y2 - y1>1400 or y2-y1<700): #1400 to 700 Before
        return False
    return True  


In [7]:
def Is_Duplicate_Id(y1,y2,id):
    global PREVIOUS_ID
    global PREVIOUS_Y1
    global PREVIOUS_Y2
    global PREVIOUS_LOCAL_IDS
    global CATTLE_LOCAL_ID
    
    try: 
        index = PREVIOUS_ID.index(id)
        print('I reached here')
        if(PREVIOUS_Y1[index]+321<=y1 and PREVIOUS_Y2[index]+371<y2): #duplicate from bottom
         #   if(id in PREVIOUS_LOCAL_IDS):
         #   #print('id: ',id,' LOCAL_ID: ',CATTLE_LOCAL_ID)
         #       return PREVIOUS_LOCAL_IDS[id][0]
            
            #print('except')
            PREVIOUS_ID.append(CATTLE_LOCAL_ID)
            PREVIOUS_Y1.append(y1)
            PREVIOUS_Y2.append(y2)
            #print('New Cattle Id')
            CATTLE_LOCAL_ID+=1
            return CATTLE_LOCAL_ID
        #elif(PREVIOUS_Y[index]+400<center): #stepping back
        #    if(id in PREVIOUS_LOCAL_IDS):
        #        return PREVIOUS_LOCAL_IDS[id][0]
        else:
            #print('Oh. here ? really?')
            PREVIOUS_Y1[index]=y1 #duplicate is solved or no duplicate and just need for last y 
            PREVIOUS_Y2[index]=y2
            #return PREVIOUS_LOCAL_IDS[index][1]
            
            #update('PREVIOUS Y')
            return PREVIOUS_ID[index]
    except:
        #print(PREVIOUS_ID)
        #print(id)
        CATTLE_LOCAL_ID += 1
        #print('except')
        PREVIOUS_ID.append(CATTLE_LOCAL_ID)
        PREVIOUS_Y1.append(y1)
        PREVIOUS_Y2.append(y2)
        return id

In [8]:

def Take_Prev_Label(y,h,id,cow_srno):
    global STORED_IDS
    global STORED_MID_Y
    global STORED_MID_Y1
    global STORED_MID_Y2
    global STORED_MISS
    global LAST_SEEN_IDS
    global LAST_SEEN_ID_CENTROIDS
    global CATTLE_LOCAL_ID
    global IS_FIRST_CATTLE 
    y1 , y2 = y , y+h
    
    if IS_FIRST_CATTLE:
        IS_FIRST_CATTLE = False
        id = CATTLE_LOCAL_ID
    #mid_y = y2
    mid_y = int(2*y + h)/2
    IS_NEW = True
    last_id = 999
    last_y1 = 0
    last_y2 = 0
    if(len(STORED_IDS)>0): 
        last_id = STORED_IDS[len(STORED_IDS)-1]
        last_y1 = STORED_MID_Y1[len(STORED_MID_Y1)-1]#max(STORED_MID_Y1)
        last_y2 = STORED_MID_Y2[len(STORED_MID_Y2)-1]#max(STORED_MID_Y2)
        MISSED_LEN = len(STORED_MISS)
        #if(IS_NEW):
        
        #    MISSED_LEN -=1
        removed = 0
        for i in range(MISSED_LEN):
            #print(i, ' missed index checking' )
            missed = STORED_MISS[i-removed]
            #print('checking ',i-removed, 'to remove')
            if(missed>70): #if missed 35 frames
    
                del STORED_MISS[i-removed]  
                del STORED_MID_Y[i-removed]
                del STORED_MID_Y1[i-removed]
                del STORED_MID_Y2[i-removed]
                del STORED_IDS[i-removed]
                removed+=1
                #print('removed')
                
    #clear misses
   
    
    threshold_1 = 200 #300
    threshold_2 = 250  #230
    Distance = 1000
     
    #if mid_y <= 1300 or mid_y >= 700:
    #    threshold_1 = 320 #350
    #    threshold_2 = 370 #280
    for i in range(1,len(STORED_MID_Y)+1):
        #print(STORED_IDS[-i-1],STORED_MID_Y[-i-1],' ',i)
        
        
        #if(STORED_MID_Y[-i]+threshold_2>=mid_y and STORED_MID_Y[-i]-threshold_1<=mid_y): # and IS_NEW): #previous 150 #200
        if(STORED_MID_Y1[-i]-threshold_1<=y1 and STORED_MID_Y1[-i]+threshold_1-50>=y1) or (STORED_MID_Y2[-i]-threshold_2<=y2 and STORED_MID_Y2[-i]+threshold_2-50>=y2): # and IS_NEW): #previous 150 #200
            if(IS_NEW):
              
                
                Distance = abs(STORED_MID_Y1[-i] - y1)
                if(abs(STORED_MID_Y2[-i] - y2)<Distance):
                    Distance = abs(STORED_MID_Y2[-i] - y2)
                IS_NEW = False
                STORED_MID_Y1[-i] = y1
                STORED_MID_Y2[-i] = y2
                
                STORED_MISS[-i]=1
                id= STORED_IDS[-i]
                #print(Distance)
                #print(id)
                
            #try:
            #    exist_index = LAST_SEEN_IDS.index(id)
            #    if(LAST_SEEN_ID_CENTROIDS[exist_index]+200>y): # showing old id
            #        LAST_SEEN_ID_CENTROIDS[exist_index] = y
            #except:
            #print('corrected id :',STORED_IDS[-i])
            elif Distance >70:
                STORED_MISS[-i]+=1
            #else:
            #    STORED_MISS[-i]-=1 #reset count to 2 when not moving
        #elif(STORED_MID_Y1[-i]<=y1 and STORED_MID_Y2[-i]>=y2):
        #        STORED_MISS[-i]=30
        else:
            STORED_MISS[-i]+=1    
     
    if(IS_NEW == False and id<last_id):
        if(y1>=last_y1 and y2>=last_y2):
            IS_NEW = True
        elif (y1>=last_y1-70 or y1<=last_y1+70) and (y2>=last_y2-70 or y2<=last_y2+70):
            return -1
    
        #elif(cow_srno==1):
    if(IS_NEW):
        CATTLE_LOCAL_ID+=1
        id=CATTLE_LOCAL_ID
        if(y1<last_y1-50 and y2<last_y2-50):
            CATTLE_LOCAL_ID-=1
            print('skipped NEW')
            for i in range(len(STORED_MID_Y)):
                STORED_MISS[i]=5
            return -1
            
        STORED_IDS.append(id)
        STORED_MID_Y.append(mid_y)
        STORED_MID_Y1.append(y1)
        STORED_MID_Y2.append(y2)
        STORED_MISS.append(1)
    #print(STORED_IDS,' IDS ',STORED_MID_Y,' SMY ',mid_y,' mid_y')
    if(IS_NEW) and False:
        #print('SMY: ',STORED_MID_Y,', new my:',mid_y) 
        #print('new id: ',id)
        updatedID = Is_Duplicate_Id(y1,y2,id)
        if(int(last_id) <int(updatedID) and y1<last_y1-150 and y2<last_y2-150): # duplicate cattle with increased cattleID
            CATTLE_LOCAL_ID-=1
            for i in range(len(STORED_MID_Y)-1,0,-1):
                STORED_MISS[i]=15
            return -1
        if(int(last_id)-1>int(updatedID)):
            return -1
            
    #if(updatedID!=id):
    #    print('orgID: ',id,' updated ID: ',updatedID)
        #id = str(updated_ID)+'_'+str(id)
        
        id=CATTLE_LOCAL_ID
        STORED_IDS.append(id)
        STORED_MID_Y.append(mid_y)
        STORED_MID_Y1.append(y1)
        STORED_MID_Y2.append(y2)
        STORED_MISS.append(1)
    
    #print('returned id :',id)
    
    #print(id)
    
    result = []
    result.append(str(id))
    print(STORED_MID_Y1,'  <===== y1, y2 =====>  ',STORED_MID_Y2,'    result =====>',result)
    #region remove stored id
    removed = 0
    #for i in range(len(STORED_MID_Y)-1,0,-1):
    #    if(y1>STORED_MID_Y1[i] and y2>STORED_MID_Y2[i]):
    #        del STORED_MISS[i-removed]  
    #        del STORED_MID_Y[i-removed]
    #        del STORED_MID_Y1[i-removed]
    #        del STORED_MID_Y2[i-removed]
    #        del STORED_IDS[i-removed]
    #        removed+=1
                 
    return result

In [9]:

def CALCULATE_MAX_CATTLE_ID(csv_path):
    print(csv_path, " is csv_path and ")

    data = pd.read_csv(csv_path)

    list_of_csv = [list(row) for row in data.values]

    prev_id_record = [] 
    prev=None

    current_cow = []
    excel_cow_count = []
    boxes = []
    file_locations = []

    for i in range (len(list_of_csv)):
        filtered_id = list_of_csv[i][0]
        actual_id = list_of_csv[i][1]
        file_locations.append(list_of_csv[i][2])
        boxes.append([list_of_csv[i][3],list_of_csv[i][4],list_of_csv[i][5],list_of_csv[i][6]])
        try: 
            index = current_cow.index(actual_id)
            excel_cow_count[index]+=1
        except:
            current_cow.append(actual_id)
            excel_cow_count.append(1)

    maxpos = excel_cow_count.index(max(excel_cow_count))
    cattle_id = current_cow[maxpos]
    return cattle_id,file_locations,boxes


In [ ]:

#filePath = 'D:\\Python\\SULarbmon\\Python\\env\\yolov8_june\\ultralytics\\runs\\segment\\sumiyoshi\\identification3\\8'
#vid_location = filePath+'/'+'vid'
#print(vid_location)
#cap = cv2.VideoWriter(vid_location,cv2.VideoWriter_fourcc(*'mp4v'), 13, (608,1080))
#writeVideo(cap,filePath)
def writeVideo(cap,filePath):
    img_array = []
    size = (608,1080)
    names = ['cow']


    vid_name = os.path.basename(os.path.normpath(filePath))
    #rint(vid_name)
    #vid_path = str(Path(filePath + "/" + vid_name ).with_suffix('.mp4'))
    id,img_locations,*xyxys = CALCULATE_MAX_CATTLE_ID(filePath+"/"+vid_name+".csv")
    #out = cv2.VideoWriter(vid_path,cv2.VideoWriter_fourcc(*'mp4v'), 6, size)
    if len(img_locations)<5: 
        return -1
    
    for ind in range(len(img_locations)):
        img = cv2.imread(img_locations[ind])
        annotator = Annotator(img, line_width=3, example=str(names))
        try:
            annotator.box_label(xyxys[0][ind],str(id), color=(15, 0, 255))
            annotated_img =cv2.resize(annotator.result(),size) 
            cap.write(annotated_img)
        except ex:
            print(skipped)
            continue
    #ut.release()
    img_array=[]
    print("done ", vid_name)
    #v2.destroyAllWindows()
    return id
print("done")
cap.release()

In [12]:
# Function to draw a bounding box and annotate the image
def draw_bounding_box(image, box, label):
    # Extract the coordinates from the box
    x1, y1, x2, y2 = box
    #print(x1,' ',y1,x2,y2)

    # Draw the bounding box rectangle on the image
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 3)

    # Define the text properties
    text = f'{label}'
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale =1.5
    thickness = 3

    # Calculate the size of the text
    (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, thickness)

    # Calculate the position for placing the text
    text_x = x1
    text_y = y1 - 10 if y1 >= 20 else y1 + 10 + text_height

    # Draw the text background rectangle
    cv2.rectangle(image, (text_x, text_y - text_height - 5), (text_x + text_width, text_y), (0, 255, 0), -1)

    # Put the label text on the image
    cv2.putText(image, text, (text_x, text_y), font, font_scale, (0, 0, 0), thickness, cv2.LINE_AA)

# Example usage



In [13]:
def overlay(image, mask, color, alpha, resize=None):
    """Combines image and its segmentation mask into a single image.
    
    Params:
        image: Training image. np.ndarray,
        mask: Segmentation mask. np.ndarray,
        color: Color for segmentation mask rendering.  tuple[int, int, int] = (255, 0, 0)
        alpha: Segmentation mask's transparency. float = 0.5,
        resize: If provided, both image and its mask are resized before blending them together.
        tuple[int, int] = (1024, 1024))

    Returns:
        image_combined: The combined image. np.ndarray

    """
    # color = color[::-1]
    colored_mask = np.expand_dims(mask, 0).repeat(3, axis=0)
    colored_mask = np.moveaxis(colored_mask, 0, -1)
    masked = np.ma.MaskedArray(image, mask=colored_mask, fill_value=color)
    image_overlay = masked.filled()

    if resize is not None:
        image = cv2.resize(image.transpose(1, 2, 0), resize)
        image_overlay = cv2.resize(image_overlay.transpose(1, 2, 0), resize)

    image_combined = cv2.addWeighted(image, 1 - alpha, image_overlay, alpha, 0)

    return image_combined

In [14]:


X1=180 #same a
X2=1600 #same as NEW_BLACK_X2 # incase of x2 out of bound
Y1=30
Y2=620
SIZE =224
Y1_NEW=180 #125  #decrease here to extend, increase to shrink 
Y2_NEW=3600  #500  # redyce here to extend , increase to do vice casa 460 previous
FRAME = 1

default=640

BATCH = 100
BATCH_COUNT = 1
PREV_BATCH = 0

LAST_SEEN = time.time()
FIRST_SEEN = True
demo_img_save_path = []

#end
prevId_record =[]
MAX_prevId = [] 
MAX_xyxy1 = [] 
MAX_xyxy2 = [] 
MAX_xyxy3 = [] 
MAX_xyxy4 = [] 
MAX_orgId = []
IMAGE_STORED_LOCATION = []
#end
prevId_record =[]

#end

#region Cattle Tracking
STORED_IDS= []
STORED_MID_Y = []
STORED_MID_Y1 = []
STORED_MID_Y2 = []
STORED_MISS = []
PREVIOUS_ID = [] # keep the record of last seen ids and position
PREVIOUS_Y1 = [] 
PREVIOUS_Y2 = [] 
PREVIOUS_LOCAL_IDS = []
CATTLE_LOCAL_ID= 0
IS_FIRST_CATTLE = True

# Predict with the model
project = 'D:/Python/SULarbmon/Python/env/yolov8_june/ultralytics/runs/segment/sumiyoshi'
name = 'identification'
dataset = "D:\\815_CowDataChecking\\sumiyoshi\\2023-07-25\\18"
#save_vid_name=  dataset.split("\\")[-1].replace('.mkv','_track')  #open this when running single video
save_vid_name = dataset.split("\\")[-1]+"_track" # open this when running multiple videos
print(save_vid_name)
results = model(dataset,imgsz=(608,1080),save=False,retina_masks=False,show=False,stream=True,device='0',conf=0.2)
#results = model('D:\\815_CowDataChecking\\20221228\\20221228_E_cow\\20221228_151533_D474.mkv',imgsz=1088,save=False,retina_masks=False,show=False,stream=True,device='0',conf=0.2)
#save_dir = increment_path(Path(project) / name, exist_ok=True)  # increment run
save_dir = increment_path(Path(project) / name,mkdir=True)
#(save_dir / 'labels' if False else save_dir).mkdir(parents=True, exist_ok=True)  # make dir
manual_cow_count = 0

#cap.set(4, 480)
save_vid_path = str(Path(os.path.join(save_dir, save_vid_name)).with_suffix('.mp4'))
print(save_vid_path)
cap = cv2.VideoWriter(save_vid_path,cv2.VideoWriter_fourcc(*'mp4v'), 13, (608,1088))

for result in results:
    #print(result.boxes)
    vid_path = result.path
    filename = vid_path.split("\\")[-1].replace(".mkv","")
    
    
    boxes = result.boxes.cpu().numpy()
    ori_img = cv2.resize(result.orig_img, (352,608), interpolation = cv2.INTER_AREA)
    annotator = Annotator(ori_img)
    box_count = 0
    cow_position = 1
    
    h,w = result.orig_shape
    count = 1
    b_boxes = []
    masks = []
    ids = []
    has_cattle = False
    if  result.masks != None:
        
        #result_masks = result.masks.cpu().numpy().masks.astype(bool)
        for m in result.masks.cpu().numpy().masks.astype(bool):
        #for i in range(1,len(result_masks)+1):
        #for m in result.masks.masks.astype(bool):
            xyxy = boxes[box_count].xyxy[0]
        #   m = result_masks[-1]
            #print(xyxy)
            #print(m.shape)
            box_count += 1
            x1= int(xyxy[0])
            y1= int(xyxy[1])
            x2= int(xyxy[2])
            y2= int(xyxy[3])

            ################## Validate  #####################
            if(check_withinROI_NEW(x1,y1,x2,y2,h,w)==False):
                print("skipped")
                continue
            #print("not skipped")
            #new_results.append(result)
            box_left = x1
            box_top = y1
            box_w = x2 - x1
            box_h = y2 - y1

            #### TRACKING
            
            ######
            new = np.zeros_like(ori_img, dtype=np.uint8)
            #print(m.shape,'  ',ori_img.shape)
            new[m] = ori_img[m]
            masks.append(m)
            
            x1= int(x1 * (352/2160))
            x2= int(x2 * (352/2160))
            y1= int(y1 * (608/3840))
            y2= int(y2 * (608/3840))

            crop = new[y1:y2, x1:x2]
            img = cv2.resize(crop, (SIZE, SIZE))
            img=img / 255.0
            
            ############# LABLE
            label = Predict_SVM(img)
            
            
            ############# TRACKING
            prev_id = Take_Prev_Label(box_top,box_h,label,cow_position)
            HAS_COW=True
            #has_cattle = True
            if(prev_id==-1): #skip cattle when prev_id // filter id is -1
                if(count==1):
                    has_seen_cattle=False
                    count-=1
                #print('skipped')
                continue
            has_cattle = True
            ids.append(prev_id[0])
            #print(prev_id)
            cow_position+=1 
            
          ######################################### 
            
            BATCH_COUNT = prev_id[0] # skip batch count here  
            #BATCH calculator
            

            ##### save crop image for writing video
            BATCH_COUNT = prev_id[0] # skip batch count here  
            #BATCH calculator
            if(FIRST_SEEN):
                LAST_SEEN = time.time() #first seen time
                FIRST_SEEN=False

            if(time.time()-LAST_SEEN>=300): # 3 mins different
                #write excel for each cattle
                # print(len(prevId_record), ' previd_record', prevId_record)
                for csv_index in range(len(prevId_record)):
                    df = pd.DataFrame(MAX_prevId[csv_index], columns = ['ID'])
                    try:
                        org_ids = torch.tensor(MAX_orgId[csv_index], device = 'cpu')
                        df["Original"] = org_ids
                    except:
                         df["Original"] = MAX_orgId[csv_index]


                    try:
                        stored_locations = torch.tensor(IMAGE_STORED_LOCATION[csv_index],device = 'cpu')
                        df["location"] = stored_locations
                    except:
                        df["location"]=IMAGE_STORED_LOCATION[csv_index]

                    df["xyxy1"] = MAX_xyxy1[csv_index]
                    df["xyxy2"] = MAX_xyxy2[csv_index]
                    df["xyxy3"] = MAX_xyxy3[csv_index]
                    df["xyxy4"] = MAX_xyxy4[csv_index]



                    now=str(datetime.now().date())

                    save_csv_each_path = str(Path(save_dir / str(prevId_record[csv_index]) / f'{str(prevId_record[csv_index])}.csv'))
                    #print(save_csv_each_path)

                    df.to_csv(save_csv_each_path, index= False) ##
                    MAX_xyxy1[csv_index]=[]
                    MAX_xyxy2[csv_index]=[]
                    MAX_xyxy3[csv_index]=[]
                    MAX_xyxy4[csv_index]=[]
                    MAX_orgId[csv_index]=[]
                    MAX_prevId[csv_index]=[]
                    IMAGE_STORED_LOCATION[csv_index]=[]

                #print("new batch")
                prevId_record = []
                MAX_prevId = []
                MAX_xyxy1 = [] 
                MAX_xyxy2 = [] 
                MAX_xyxy3 = [] 
                MAX_xyxy4 = [] 
                MAX_orgId = [] 
                IMAGE_STORED_LOCATION = []


                cattle_ids = []

            LAST_SEEN = time.time()
            
            ### annotate ######
            b = xyxy  # get box coordinates in (top, left, bottom, right) format
            #c = box.cls
            #annotator.box_label(b, str(prev_id[0]))
            b_boxes.append([x1,y1,x2,y2])
            

             ###################### CREATE dir to save img
            base_path = str(Path(save_dir / prev_id[0]))
            if not os.path.exists(base_path):
                os.makedirs(base_path)

            LAST_SEEN = time.time()

            FRAME+=1
            #save each images for taking max label later
            ######################################

            demo_annotated_img_save_path = Path(base_path+ '/' + f'{filename}_{str(manual_cow_count).zfill(4)}.jpg')
            try:
                index_prevId = prevId_record.index(int(prev_id[0]))
                #print(index_prevId)
                MAX_prevId[index_prevId].append(int(prev_id[0]))#,int(label[0]),xyxy)
                MAX_xyxy1[index_prevId].append(x1)
                MAX_xyxy2[index_prevId].append(y1)
                MAX_xyxy3[index_prevId].append(x2)
                MAX_xyxy4[index_prevId].append(y2)
                MAX_orgId[index_prevId].append(label[0])

                IMAGE_STORED_LOCATION[index_prevId].append(demo_annotated_img_save_path)

            except :
                prevId_record.append(int(prev_id[0]))
                # print(len(prevId_record)-1, 'prevID_record ', len(MAX_prevId) , 'max_previd' )

                #MAX_prevId[len(prevId_record)-1].append(int(prev_id[0]))#,int(label[0]),xyxy)
                #MAX_xyxy[len(MAX_prevId)-1].append(xyxy)
                #MAX_orgId[len(MAX_prevId)-1].append(int(label[0]))
                MAX_prevId.append([int(prev_id[0])])#,int(label[0]),xyxy)

                MAX_xyxy1.append([x1])
                MAX_xyxy2.append([y1])
                MAX_xyxy3.append([x2])
                MAX_xyxy4.append([y2])
                MAX_orgId.append([label[0]])
                IMAGE_STORED_LOCATION.append([demo_annotated_img_save_path])

            try:
                #demo_vid_index = demo_vid_path.index(demo_path)
                demo_vid_index = demo_img_save_path.index(base_path)

                #print("path exist")
            except:
                #manual_summarize_ids.append(int(prev_id[0]))
                #manual_local_ids.append(manual_id)
                #manual_id +=1
                demo_img_save_path.append(base_path)
                
            try:
                ori_img = overlay(ori_img,m,(0,0,255),0.3)
                cv2.imwrite(str(demo_annotated_img_save_path), ori_img) #save ori_img
                #print(demo_annotated_img_save_path)
            except:
                print('cannot save ',demo_annotated_img_save_path)
            #change cropped size here  #230 to 215 410 to 390

            manual_cow_count += 1


    #frame = annotator.result() 
    frame = cv2.resize(ori_img, (608,1088), interpolation = cv2.INTER_AREA)
    if has_cattle:
        #ori_img = cv2.resize(ori_img,(1088,1088))
        for i in range(len(b_boxes)):
            box = b_boxes[i]
            prev_id = ids[i]
            draw_bounding_box(ori_img,(box[0],box[1],box[2],box[3]),str(prev_id))
            ori_img = overlay(ori_img,masks[i],(0,0,255),0.3)
        frame = cv2.resize(ori_img, (608,1088), interpolation = cv2.INTER_AREA)
        cap.write(frame)
    
    
    cv2.imshow('YOLO V8 Detection', cv2.resize(ori_img, (485,870), interpolation = cv2.INTER_AREA))     
    if cv2.waitKey(1) & 0xFF == ord(' '):
        break


###### write final csv
for csv_index in range(len(prevId_record)):
    df = pd.DataFrame(MAX_prevId[csv_index] , columns = ['ID'])
    try:
        org_ids = torch.tensor(MAX_orgId[csv_index], device = 'cpu')
        df["Original"] = org_ids
    except:
        df["Original"] = MAX_orgId[csv_index]

    try:
        stored_locations = torch.tensor(IMAGE_STORED_LOCATION[csv_index],device = 'cpu')
        df["location"] = stored_locations
    except:
        df["location"]=IMAGE_STORED_LOCATION[csv_index]
    df["xyxy1"] = MAX_xyxy1[csv_index]
    df["xyxy2"] = MAX_xyxy2[csv_index]
    df["xyxy3"] = MAX_xyxy3[csv_index]
    df["xyxy4"] = MAX_xyxy4[csv_index]


    now=str(datetime.now().date())

    save_csv_each_path = str(Path(save_dir / str(prevId_record[csv_index]) / f'{str(prevId_record[csv_index])}.csv'))
    df.to_csv(save_csv_each_path, index= False)##asdfasdf
prevId_record = []
MAX_prevId = []
MAX_xyxy1 = [] 
MAX_xyxy2 = [] 
MAX_xyxy3 = [] 
MAX_xyxy4 = [] 
MAX_orgId = [] 
IMAGE_STORED_LOCATION = []

cattle_ids = []
#################################\
manual_summarize_ids = []
manual_local_ids = []

manual_summarize_ids = []
manual_local_ids = []
#### write video after saving csv
final_cattle_count = 1

cap.release() #tracking video
cv2.destroyAllWindows()

save_vid_name=  dataset.split("\\")[-1].replace('.mkv','')+'_classification'
save_vid_path = str(Path(os.path.join(save_dir, save_vid_name)).with_suffix('.mp4'))
classification_vid = cv2.VideoWriter(save_vid_path,cv2.VideoWriter_fourcc(*'mp4v'), 13, (608,1080))

for loc in range(len(demo_img_save_path)):
    print(demo_img_save_path[loc])
    final_cattle_id = writeVideo(classification_vid,demo_img_save_path[loc])
    if(final_cattle_id != -1):
        manual_local_ids.append(final_cattle_count)
        manual_summarize_ids.append(final_cattle_id)
        final_cattle_count+=1 

classification_vid.release() #tracking video
print(save_vid_path)
#### write video after saving csv




WARNING  imgsz=[608, 1080] must be multiple of max stride 32, updating to [608, 1088]


18_track
yolo->v8->segement->model.py 235
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\sumiyoshi\identification12\18_track.mp4
I am in stream_loaders.py 168


video 1/2 (1/6016) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-180000-180501.mp4: 608x352 (no detections), 42.0ms
video 1/2 (2/6016) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-180000-180501.mp4: 608x352 (no detections), 35.0ms
video 1/2 (3/6016) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-180000-180501.mp4: 608x352 (no detections), 35.0ms
video 1/2 (4/6016) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-180000-180501.mp4: 608x352 (no detections), 35.0ms
video 1/2 (5/6016) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-180000-180501.mp4: 608x352 (no detections), 34.0ms
video 1/2 (6/6016) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-180000-180501.mp4: 608x352 (no detections), 35.0ms
video 1/2 (7/6016) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-180000-180501.mp4: 608x352 (no detections), 35.0ms
video 1/2 (8/6016) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-180000-180501.mp4: 608x352 (no detections),

skipped


video 1/2 (548/6016) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-180000-180501.mp4: 608x352 (no detections), 20.0ms
video 1/2 (549/6016) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-180000-180501.mp4: 608x352 (no detections), 20.0ms
video 1/2 (550/6016) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-180000-180501.mp4: 608x352 (no detections), 22.0ms
WARNING  'Masks.masks' is deprecated. Use 'Masks.data' instead.
video 1/2 (551/6016) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-180000-180501.mp4: 608x352 1 cow, 18.0ms
WARNING  'Masks.masks' is deprecated. Use 'Masks.data' instead.
video 1/2 (552/6016) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-180000-180501.mp4: 608x352 1 cow, 19.0ms
WARNING  'Masks.masks' is deprecated. Use 'Masks.data' instead.
video 1/2 (553/6016) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-180000-180501.mp4: 608x352 1 cow, 18.7ms


...

skipped
skipped
skipped
skipped
skipped


WARNING  'Masks.masks' is deprecated. Use 'Masks.data' instead.
video 2/2 (5688/5984) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-181501-182000.mp4: 608x352 2 cows, 19.0ms
video 2/2 (5689/5984) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-181501-182000.mp4: 608x352 (no detections), 20.0ms
video 2/2 (5690/5984) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-181501-182000.mp4: 608x352 (no detections), 22.0ms


skipped
skipped


video 2/2 (5691/5984) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-181501-182000.mp4: 608x352 (no detections), 21.0ms
video 2/2 (5692/5984) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-181501-182000.mp4: 608x352 (no detections), 20.0ms
video 2/2 (5693/5984) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-181501-182000.mp4: 608x352 (no detections), 16.9ms
video 2/2 (5694/5984) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-181501-182000.mp4: 608x352 (no detections), 19.0ms
video 2/2 (5695/5984) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-181501-182000.mp4: 608x352 (no detections), 18.0ms
video 2/2 (5696/5984) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-181501-182000.mp4: 608x352 (no detections), 21.0ms
video 2/2 (5697/5984) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-181501-182000.mp4: 608x352 (no detections), 21.0ms
video 2/2 (5698/5984) D:\815_CowDataChecking\sumiyoshi\2023-07-25\18\20230725-181501-182000.mp4: 

D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\sumiyoshi\identification12\1
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\sumiyoshi\identification12\1/1.csv  is csv_path and 
done  1
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\sumiyoshi\identification12\2
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\sumiyoshi\identification12\2/2.csv  is csv_path and 
done  2
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\sumiyoshi\identification12\3
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\sumiyoshi\identification12\3/3.csv  is csv_path and 
done  3
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\sumiyoshi\identification12\4
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\sumiyoshi\identification12\4/4.csv  is csv_path and 
done  4
D:\Python\SULarbmon\Python\env\yolov8_june\ultralytics\runs\segment\sumiyoshi\identification12\5

In [ ]:
cap.release()
cv2.destroyAllWindows()


##### FOR DETECTION + TRACKING + IDENTIFICAION THAT WILL COME LATER

X1=200 #same as NEW_BLACK_X1
X2=400 #same as NEW_BLACK_X2 # incase of x2 out of bound
Y1=120
Y2=500
SIZE =224
Y1_NEW=120 #125  #decrease here to extend, increase to shrink 
Y2_NEW=510  #500  # redyce here to extend , increase to do vice casa 460 previous
FRAME = 1

default=640

BATCH = 100
BATCH_COUNT = 1
PREV_BATCH = 0

LAST_SEEN = time.time()
FIRST_SEEN = True
demo_img_save_path = []

prevId_record =[]
MAX_prevId = [] 
MAX_xyxy1 = [] 
MAX_xyxy2 = [] 
MAX_xyxy3 = [] 
MAX_xyxy4 = [] 
MAX_orgId = []
IMAGE_STORED_LOCATION = []
#end
prevId_record =[]

#end

#region Cattle Tracking
STORED_IDS= []
STORED_MID_Y = []
STORED_MID_Y1 = []
STORED_MID_Y2 = []
STORED_MISS = []
PREVIOUS_ID = [] # keep the record of last seen ids and position
PREVIOUS_Y1 = [] 
PREVIOUS_Y2 = [] 
PREVIOUS_LOCAL_IDS = []
CATTLE_LOCAL_ID= 1
IS_FIRST_CATTLE = True
# Predict with the model
project = 'D:/Python/SULarbmon/Python/env/yolov8_june/ultralytics/runs/segment/honkawa'
name = 'predict-vgg-svm'
dataset = "D:\\815_CowDataChecking\\sumiyoshi\\2023-07-21\\09"
#save_vid_name=  dataset.split("\\")[-1].replace('.mkv','_track')  #open this when running single video
save_vid_name = dataset.split("\\")[-1]+"_track" # open this when running multiple videos
print(save_vid_name)
results = model(dataset,imgsz=(1920,1080),save=False,retina_masks=False,show=False,stream=True,device='0',conf=0.3)
#results = model('D:\\815_CowDataChecking\\20221228\\20221228_E_cow\\20221228_151533_D474.mkv',imgsz=1088,save=False,retina_masks=False,show=False,stream=True,device='0',conf=0.2)
#save_dir = increment_path(Path(project) / name, exist_ok=True)  # increment run
save_dir = increment_path(Path(project) / name,mkdir=True)
#(save_dir / 'labels' if False else save_dir).mkdir(parents=True, exist_ok=True)  # make dir
manual_cow_count = 0

#cap.set(4, 480)
save_vid_path = str(Path(os.path.join(save_dir, save_vid_name)).with_suffix('.mp4'))
print(save_vid_path)
cap = cv2.VideoWriter(save_vid_path,cv2.VideoWriter_fourcc(*'mp4v'), 13, (1920,1080))

for result in results:
    #print(result.boxes)
    vid_path = result.path
    filename = vid_path.split("\\")[-1].replace(".mkv","")
    
    
    boxes = result.boxes.cpu().numpy()
    ori_img = cv2.resize(result.orig_img, (1920,1088), interpolation = cv2.INTER_AREA)
    annotator = Annotator(ori_img)
    box_count = 0
    cow_position = 1
    
    h,w = result.orig_shape
    count = 1
    b_boxes = []
    masks = []
    ids = []
    has_cattle = False
    if  result.masks != None:
        
        #result_masks = result.masks.cpu().numpy().masks.astype(bool)
        for m in result.masks.cpu().numpy().masks.astype(bool):
        #for i in range(1,len(result_masks)+1):
        #for m in result.masks.masks.astype(bool):
            xyxy = boxes[box_count].xyxy[0]
        #   m = result_masks[-1]
            #print(xyxy)
            #print(m.shape)
            box_count += 1
            x1= int(xyxy[0])
            y1= int(xyxy[1])
            x2= int(xyxy[2])
            y2= int(xyxy[3])

            ################## Validate  #####################
            if(check_withinROI_NEW(x1,y1,x2,y2,h,w)==False):
                continue
            #print("not skipped")
            #new_results.append(result)
            box_left = x1
            box_top = y1
            box_w = x2 - x1
            box_h = y2 - y1

            new = np.zeros_like(ori_img, dtype=np.uint8)
            new[m] = ori_img[m]
            #masks.append(m)
            
            x1= int(x1 * (1088/1920))
            x2= int(x2 * (1088/1920))
            #y1= int(y1 * (1088/2992))
            #y2= int(y2 * (1088/2992))

            crop = new[y1:y2, x1:x2]
            ###### LABEL
            img = cv2.resize(crop, (SIZE, SIZE))
            img=img / 255.0
            label = Predict_SVM(img)
            
            prev_id = Take_Prev_Label(box_top,box_h,label,cow_position)
          #########################################  


            HAS_COW=True
            #has_cattle = True
            if(prev_id==-1): #skip cattle when prev_id // filter id is -1
                if(count==1):
                    has_seen_cattle=False
                    count-=1
                #print('skipped')
                continue
            has_cattle = True
            ids.append(prev_id[0])
            #print(prev_id)
            cow_position+=1 
            BATCH_COUNT = prev_id[0] # skip batch count here  
            #BATCH calculator
            

            ##### save crop image for writing video
            BATCH_COUNT = prev_id[0] # skip batch count here  
            #BATCH calculator
            if (FIRST_SEEN == True):
                LAST_SEEN = time.time() #first seen time
                FIRST_SEEN=False

            if(time.time()-LAST_SEEN>=300): # 3 mins different
                #write excel for each cattle
                # print(len(prevId_record), ' previd_record', prevId_record)
                for csv_index in range(len(prevId_record)):
                    df = pd.DataFrame(MAX_prevId[csv_index], columns = ['ID'])
                    try:
                        org_ids = torch.tensor(MAX_orgId[csv_index], device = 'cpu')
                        df["Original"] = org_ids
                    except:
                         df["Original"] = MAX_orgId[csv_index]


                    try:
                        stored_locations = torch.tensor(IMAGE_STORED_LOCATION[csv_index],device = 'cpu')
                        df["location"] = stored_locations
                    except:
                        df["location"]=IMAGE_STORED_LOCATION[csv_index]

                    df["xyxy1"] = MAX_xyxy1[csv_index]
                    df["xyxy2"] = MAX_xyxy2[csv_index]
                    df["xyxy3"] = MAX_xyxy3[csv_index]
                    df["xyxy4"] = MAX_xyxy4[csv_index]



                    now=str(datetime.now().date())

                    save_csv_each_path = str(Path(save_dir / str(prevId_record[csv_index]) / f'{str(prevId_record[csv_index])}.csv'))
                    #print(save_csv_each_path)

                    df.to_csv(save_csv_each_path, index= False) ##
                    MAX_xyxy1[csv_index]=[]
                    MAX_xyxy2[csv_index]=[]
                    MAX_xyxy3[csv_index]=[]
                    MAX_xyxy4[csv_index]=[]
                    MAX_orgId[csv_index]=[]
                    MAX_prevId[csv_index]=[]
                    IMAGE_STORED_LOCATION[csv_index]=[]

                #print("new batch")
                prevId_record = []
                MAX_prevId = []
                MAX_xyxy1 = [] 
                MAX_xyxy2 = [] 
                MAX_xyxy3 = [] 
                MAX_xyxy4 = [] 
                MAX_orgId = [] 
                IMAGE_STORED_LOCATION = []


                cattle_ids = []

            LAST_SEEN = time.time()
            
            
            ######
            
            
            ### annotate ######
            b = xyxy  # get box coordinates in (top, left, bottom, right) format
            #c = box.cls
            #annotator.box_label(b, str(prev_id[0]))
            b_boxes.append([x1,y1,x2,y2])
            

            ###################### CREATE dir to save img
            base_path = str(Path(save_dir / prev_id[0]))
            if not os.path.exists(base_path):
                os.makedirs(base_path)

            LAST_SEEN = time.time()

            FRAME+=1
            #save each images for taking max label later
            ######################################

            demo_annotated_img_save_path = Path(save_dir+ '/' + f'{filename}_{str(manual_cow_count).zfill(4)}.jpg')
            try:
                index_prevId = prevId_record.index(int(prev_id[0]))
                #print(index_prevId)
                MAX_prevId[index_prevId].append(int(prev_id[0]))#,int(label[0]),xyxy)
                MAX_xyxy1[index_prevId].append(x1)
                MAX_xyxy2[index_prevId].append(y1)
                MAX_xyxy3[index_prevId].append(x2)
                MAX_xyxy4[index_prevId].append(y2)
                MAX_orgId[index_prevId].append(label[0])

                IMAGE_STORED_LOCATION[index_prevId].append(demo_annotated_img_save_path)

            except :
                prevId_record.append(int(prev_id[0]))
                # print(len(prevId_record)-1, 'prevID_record ', len(MAX_prevId) , 'max_previd' )

                #MAX_prevId[len(prevId_record)-1].append(int(prev_id[0]))#,int(label[0]),xyxy)
                #MAX_xyxy[len(MAX_prevId)-1].append(xyxy)
                #MAX_orgId[len(MAX_prevId)-1].append(int(label[0]))
                MAX_prevId.append([int(prev_id[0])])#,int(label[0]),xyxy)

                MAX_xyxy1.append([x1])
                MAX_xyxy2.append([y1])
                MAX_xyxy3.append([x2])
                MAX_xyxy4.append([y2])
                MAX_orgId.append([label[0]])
                IMAGE_STORED_LOCATION.append([demo_annotated_img_save_path])

            try:
                #demo_vid_index = demo_vid_path.index(demo_path)
                demo_vid_index = demo_img_save_path.index(base_path)

                #print("path exist")
            except:
                #manual_summarize_ids.append(int(prev_id[0]))
                #manual_local_ids.append(manual_id)
                #manual_id +=1
                demo_img_save_path.append(base_path)
                
            try:
                ori_img = overlay(ori_img,m,(0,0,255),0.3)
                cv2.imwrite(str(demo_annotated_img_save_path), ori_img) #save ori_img
                #print(demo_annotated_img_save_path)
            except:
                print('cannot save ',demo_annotated_img_save_path)
            #change cropped size here  #230 to 215 410 to 390

            manual_cow_count += 1

    #frame = annotator.result() 
    frame = cv2.resize(ori_img, (1080,1080), interpolation = cv2.INTER_AREA)
    if has_cattle:
        #ori_img = cv2.resize(ori_img,(1088,1088))
        for i in range(len(b_boxes)):
            box = b_boxes[i]
            #mask = masks[i]
            prev_id = ids[i]
            draw_bounding_box(ori_img,(box[0],box[1],box[2],box[3]),str(prev_id))
            #ori_img = overlay(ori_img,mask,(0,0,255),0.3)
        frame = cv2.resize(ori_img, (1080,1080), interpolation = cv2.INTER_AREA)
        cap.write(frame)
    
    
    cv2.imshow('YOLO V8 Detection', frame)     
    if cv2.waitKey(1) & 0xFF == ord(' '):
        break

###### write final csv
for csv_index in range(len(prevId_record)):
    df = pd.DataFrame(MAX_prevId[csv_index] , columns = ['ID'])
    try:
        org_ids = torch.tensor(MAX_orgId[csv_index], device = 'cpu')
        df["Original"] = org_ids
    except:
        df["Original"] = MAX_orgId[csv_index]

    try:
        stored_locations = torch.tensor(IMAGE_STORED_LOCATION[csv_index],device = 'cpu')
        df["location"] = stored_locations
    except:
        df["location"]=IMAGE_STORED_LOCATION[csv_index]
    df["xyxy1"] = MAX_xyxy1[csv_index]
    df["xyxy2"] = MAX_xyxy2[csv_index]
    df["xyxy3"] = MAX_xyxy3[csv_index]
    df["xyxy4"] = MAX_xyxy4[csv_index]


    now=str(datetime.now().date())

    save_csv_each_path = str(Path(save_dir / str(prevId_record[csv_index]) / f'{str(prevId_record[csv_index])}.csv'))
    df.to_csv(save_csv_each_path, index= False)##asdfasdf
prevId_record = []
MAX_prevId = []
MAX_xyxy1 = [] 
MAX_xyxy2 = [] 
MAX_xyxy3 = [] 
MAX_xyxy4 = [] 
MAX_orgId = [] 
IMAGE_STORED_LOCATION = []

cattle_ids = []
#################################\
manual_summarize_ids = []
manual_local_ids = []

manual_summarize_ids = []
manual_local_ids = []
#### write video after saving csv
final_cattle_count = 1

cap.release() #tracking video
cv2.destroyAllWindows()

save_vid_name=  dataset.split("\\")[-1].replace('.mkv','')+'_classification'
save_vid_path = str(Path(os.path.join(save_dir, save_vid_name)).with_suffix('.mp4'))
classification_vid = cv2.VideoWriter(save_vid_path,cv2.VideoWriter_fourcc(*'mp4v'), 13, (1080,1080))

for loc in range(len(demo_img_save_path)):
    print(demo_img_save_path[loc])
    final_cattle_id = writeVideo(classification_vid,demo_img_save_path[loc])
    if(final_cattle_id != -1):
        manual_local_ids.append(final_cattle_count)
        manual_summarize_ids.append(final_cattle_id)
        final_cattle_count+=1 

classification_vid.release() #tracking video
print(save_vid_path)
#### write video after saving csv

